In [82]:
import nltk
import pandas as pd

In [119]:
df_original = pd.read_csv('../OnlineHarassmentDataset/onlineHarassmentDataset.tdf',sep='\t', lineterminator='\r',header = 'infer', encoding='latin-1')
df=df_original

In [120]:
df.head()

,ID,Code,Tweet,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,H,@Grumpy_P_Sloth @deanesmay feminists argue for...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,H,1948Army of England helped the Jews to occupy ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,H,Dutch Leader Says Europe to collapse In 6 Week...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,H,RT @__DeLay: The next day the Romans and the J...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,H,RT @Bakersman_Joe: When Hitler Invited The Jew...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
# How balanced are the classes?
print('Harassment is ' + str(round(sum(df['Code'] == 'H')/df.shape[0],4) * 100) + '% of the dataset')

Harassment is 25.96% of the dataset


In [122]:
# Quality of Life renaming
df = df[['Code','Tweet']]
df.rename(index=str, columns={"Code": "code", "Tweet": "tweet"}, inplace = True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [123]:
df.iloc[200,1]

'@azhodzic lmfaaaooooo @jeffbarnes_  you fucking nigger'

# List of Regex Strings to Remove

In [124]:
df['tweet'] = df['tweet'].apply(lambda tweet: tweet.lower())

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [125]:
regex_queries = [
                 '@([A-Za-z0-9_]+)', #Usernames
                'rt\s:\s', #Retweets
                 '(https?:\s?\/\s?\/)(\s)?(www\.)?(\s?)(\w+\.)*([\w\-\s]+\/)*([\w-]+)\/?' # Hyperlinks
                ]

In [126]:
df['tweet'].replace(regex_queries,'',regex = True, inplace = True)

/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [127]:
df['tweet'].iloc[2]

'dutch leader says europe to collapse in 6 weeks thnks > , , &  #altright #whitegenocide '

In [129]:
df_original['tweet'].iloc[2]

KeyError: 'tweet'

In [128]:
df.values

array([['H',
        "  feminists argue for raising minimum wage cause a women's studies major needs the $15 an hour wage at her mcjob."],
       ['H',
        '1948army of england helped the jews to occupy palestine.after that the jews killed thousands of englishmen and returned to england as dogs1'],
       ['H',
        'dutch leader says europe to collapse in 6 weeks thnks > , , &  #altright #whitegenocide '],
       ...,
       ['H',
        'you fucking nigger you did it to yourself you fucking nigger you killed yourself you fucking nigger believe its happening you fucking nigger'],
       ['H',
        'young white woman gunned down by black beast in norcal #whitelivesmatter '],
       ['H',
        'your descendants, the jews, will be doomed to descend from the throne of power,']],
      dtype=object)